# What am I listening on Spotify?

In this project, I'll take a look at my Spotify Straming History through out 2020-2021.
If you didn't know, you can request your personal data to Spotify!

In [12]:
# Let's import some packages
import pandas as pd
import numpy as np
import plotly.express as px

from datetime import datetime

In my case, I have one json contaning all my streaming history (if you listen to more music than me you might have more than one file)

I also received the tracks file, which contains the albums of the songs. 
I'll use the pandas to read the data from the json files.

In [2]:
df_streaming = pd.read_json('MyData/StreamingHistory0.json')

df_track = pd.read_json('MyData/tracks.json')

In [13]:
# Taking a look at the head of the file
# The amount of time spent listening to one track is in miliseconds, we can convert this column later
df_streaming.head()

,endTime,artistName,trackName,msPlayed
0,2020-09-16 22:59,Mark Maxwell,Sweetness,199850
1,2020-09-17 13:37,Anne Reburn,The Longest Time,15298
2,2020-09-21 20:07,Donny Osmond,"I'll Make a Man Out of You - From ""Mulan""/Soun...",83935
3,2020-09-23 12:48,Gorillaz,Strange Timez (feat. Robert Smith),29937
4,2020-09-23 12:49,Welcome to Night Vale,New to Night Vale? Start here!,66938


In [14]:
df_streaming.describe(include = "all")

,endTime,artistName,trackName,msPlayed
count,3436,3436,3436,3.436000e+03
unique,2021,562,1116,NaN
top,2021-06-20 22:11,BRADIO,幸せのシャナナ,NaN
freq,100,279,22,NaN
mean,NaN,NaN,NaN,1.088752e+05
std,NaN,NaN,NaN,1.689687e+05
min,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,NaN,2.557750e+03
50%,NaN,NaN,NaN,3.089350e+04
75%,NaN,NaN,NaN,2.044170e+05


In [15]:
#df_track['track:uri'] = df_track['uri'].str.split('spotify:').str[1]
df_track.head()

,artist,album,track,uri
0,Panic! At The Disco,"Too Weird to Live, Too Rare to Die!",Girls / Girls / Boys,spotify:track:2wEjPgOCEczmjlvq60LyQi
1,BRADIO,Flyers,Flyers,spotify:track:52sk8TPaHcLQVCjFaeRI6t
2,RADWIMPS,君の名は。,三葉のテーマ,spotify:track:5zEKHWr92eqOaoCgmELP8k
3,Hello Sleepwalkers,Masked Monkey Awakening,Tenchi Souzou,spotify:track:4PGluOFvKJx9kTtAbOeVJl
4,Rainbow Dash,It's a Pony Kind of Christmas,Jingle Bells,spotify:track:2Y42Fkup0CVfrNtxNUdz5f


In [6]:
df_track.describe()

,artist,album,track,uri,UniqueID,track:uri
count,385,385,385,385,385,385
unique,66,55,385,385,385,385
top,Martin Baltscheit,A Million Ways to Die in the West,Girls / Girls / Boys,spotify:track:2wEjPgOCEczmjlvq60LyQi,Panic! At The Disco:Girls / Girls / Boys,track:2wEjPgOCEczmjlvq60LyQi
freq,57,57,1,1,1,1


In [3]:
# Here I'm creating a columns with the names of the artist and the track so I can emerge both DFs 
df_streaming['UniqueID'] = df_streaming['artistName'] + ':' + df_streaming['trackName']

df_track['UniqueID'] = df_track['artist'] + ':' + df_track['track']

In [4]:
df_merged = pd.merge(df_streaming, df_track[["album", "UniqueID"]], how = "left", on = ["UniqueID"])

Let's see what Yhe final product

In [5]:
df_merged.head()

,endTime,artistName,trackName,msPlayed,UniqueID,album
0,2020-09-16 22:59,Mark Maxwell,Sweetness,199850,Mark Maxwell:Sweetness,NaN
1,2020-09-17 13:37,Anne Reburn,The Longest Time,15298,Anne Reburn:The Longest Time,The Longest Time
2,2020-09-21 20:07,Donny Osmond,"I'll Make a Man Out of You - From ""Mulan""/Soun...",83935,Donny Osmond:I'll Make a Man Out of You - From...,NaN
3,2020-09-23 12:48,Gorillaz,Strange Timez (feat. Robert Smith),29937,Gorillaz:Strange Timez (feat. Robert Smith),NaN
4,2020-09-23 12:49,Welcome to Night Vale,New to Night Vale? Start here!,66938,Welcome to Night Vale:New to Night Vale? Start...,NaN


In [6]:
# Converting the time Played to hours
df_merged["HoursPlayed"] = (df_merged["msPlayed"]/(1000*60*60))%24
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3436 entries, 0 to 3435
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   endTime      3436 non-null   object 
 1   artistName   3436 non-null   object 
 2   trackName    3436 non-null   object 
 3   msPlayed     3436 non-null   int64  
 4   UniqueID     3436 non-null   object 
 5   album        308 non-null    object 
 6   HoursPlayed  3436 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 214.8+ KB


In [7]:
grouped_artists = df_merged.groupby("artistName")
artists = df_merged["artistName"].unique()
dummy_list =  []
for artist in artists:
    if grouped_artists.get_group(artist)["HoursPlayed"].sum() > 0.8:
        dummy_list.append(grouped_artists.get_group(artist))
    
df_merged_copy = pd.concat(dummy_list, ignore_index=True)                              

In [17]:
fig = px.histogram(df_merged_copy, x = "artistName", y = "HoursPlayed", opacity=0.8)
fig.show()

In [18]:
fig = px.scatter(df_merged_copy, x = "endTime", y = "HoursPlayed")
fig.show()